In [1]:
# !pip install -U gensim

In [6]:
!pip install pycocoevalcap --user
# !pip install pycocotools --user
# !pip install rouge-score --user
# !pip install rouge --user
# !pip install git+https://github.com/Maluuba/nlg-eval.git@master --user

  Using cached pycocoevalcap-1.2-py3-none-any.whl (104.3 MB)
  Using cached pycocotools-2.0.6.tar.gz (24 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  ERROR: Command errored out with exit status 1:
   command: /usr/local/bin/python /usr/local/lib/python3.8/site-packages/pip/_vendor/pep517/in_process/_in_process.py build_wheel /tmp/tmp2iqxfyua
       cwd: /tmp/pip-install-mzgvaf9i/pycocotools_41a2242966f64a15b356556c63b701fa
  Complete output (20 lines):
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build/lib.linux-x86_64-cpython-38
  creating build/lib.linux-x86_64-cpython-38/pycocotools
  copying pycocotools/mask.py -> build/lib.linux-x86_64-cpython-38/pycocotools
  copying pycocotools/cocoeval.py -> build/lib.linux-x86_64-cpython-38/pycocotools
  copying pycocotools/coco.py -> build/lib.linux-x86_64-cpython-38/pycocotools
  copying pycocotools/__init__

In [2]:
import pandas as pd
import numpy as np
from collections import Counter
# from gensim.utils import tokenize, simple_preprocess, simple_tokenize
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
import pickle

[nltk_data] Downloading package punkt to /home/ss4yd/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
df = pd.read_csv('../prepared_prelim_data.csv')
df.head()

,pid,tissue_type,notes,svs_path,patch_path,reps_path
0,GTEX-1117F-0726,Heart - Atrial Appendage,"2 pieces, no abnormalities",/project/GutIntelligenceLab/ss4yd/gtex_data/ac...,/project/GutIntelligenceLab/ss4yd/gtex_data/pa...,/project/GutIntelligenceLab/ss4yd/gtex_data/hi...
1,GTEX-1117F-1026,Lung,"2 pieces, moderate congestion/moderate to mark...",/project/GutIntelligenceLab/ss4yd/gtex_data/ac...,/project/GutIntelligenceLab/ss4yd/gtex_data/pa...,/project/GutIntelligenceLab/ss4yd/gtex_data/hi...
2,GTEX-1117F-1126,Liver,"2 pieces, subtotal massive hepatic necrosis",/project/GutIntelligenceLab/ss4yd/gtex_data/ac...,/project/GutIntelligenceLab/ss4yd/gtex_data/pa...,/project/GutIntelligenceLab/ss4yd/gtex_data/hi...
3,GTEX-1117F-1226,Spleen,"2 pieces, marked congestion/autolysis",/project/GutIntelligenceLab/ss4yd/gtex_data/ac...,/project/GutIntelligenceLab/ss4yd/gtex_data/pa...,/project/GutIntelligenceLab/ss4yd/gtex_data/hi...
4,GTEX-1117F-1326,Adipose - Visceral (Omentum),"2 pieces, diffuse mesothelial hyperplasia; ~10...",/project/GutIntelligenceLab/ss4yd/gtex_data/ac...,/project/GutIntelligenceLab/ss4yd/gtex_data/pa...,/project/GutIntelligenceLab/ss4yd/gtex_data/hi...


In [4]:
df['tokens']=df.notes.apply(word_tokenize)

In [5]:
max_len=max(df.tokens.apply(len))
max_len

41

In [6]:
captions = []
word_freq = Counter()
for index, row in df.iterrows():
    # Update word frequency
    word_freq.update(row['tokens'])
    if len(row['tokens']) <= max_len:
        captions.append(row['tokens'])

In [7]:
min_word_freq=1
words = [w for w in word_freq.keys() if word_freq[w] > min_word_freq]
word_map = {k: v + 1 for v, k in enumerate(words)}
word_map['<unk>'] = len(word_map)+1
word_map['<start>'] = len(word_map)+1
word_map['<end>'] = len(word_map)+1
word_map['<pad>'] = 0

In [8]:
def add_start_end(ls,max_len):
    ls = ['<start>']+ls+['<end>']+['<pad>']*(max_len-len(ls))
    return ls

df['p_tokens']=df.tokens.apply(lambda x: add_start_end(x,max_len))

In [9]:
df.head()['p_tokens'][0]

['<start>',
 '2',
 'pieces',
 ',',
 'no',
 'abnormalities',
 '<end>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>']

In [10]:
with open('word_map.pickle', 'wb') as file:
    pickle.dump(word_map, file)

In [11]:
len(word_map)

651

In [12]:
df['caplens']=df.tokens.apply(lambda x: len(x)+2)
df

,pid,tissue_type,notes,svs_path,patch_path,reps_path,tokens,p_tokens,caplens
0,GTEX-1117F-0726,Heart - Atrial Appendage,"2 pieces, no abnormalities",/project/GutIntelligenceLab/ss4yd/gtex_data/ac...,/project/GutIntelligenceLab/ss4yd/gtex_data/pa...,/project/GutIntelligenceLab/ss4yd/gtex_data/hi...,"[2, pieces, ,, no, abnormalities]","[<start>, 2, pieces, ,, no, abnormalities, <en...",7
1,GTEX-1117F-1026,Lung,"2 pieces, moderate congestion/moderate to mark...",/project/GutIntelligenceLab/ss4yd/gtex_data/ac...,/project/GutIntelligenceLab/ss4yd/gtex_data/pa...,/project/GutIntelligenceLab/ss4yd/gtex_data/hi...,"[2, pieces, ,, moderate, congestion/moderate, ...","[<start>, 2, pieces, ,, moderate, congestion/m...",10
2,GTEX-1117F-1126,Liver,"2 pieces, subtotal massive hepatic necrosis",/project/GutIntelligenceLab/ss4yd/gtex_data/ac...,/project/GutIntelligenceLab/ss4yd/gtex_data/pa...,/project/GutIntelligenceLab/ss4yd/gtex_data/hi...,"[2, pieces, ,, subtotal, massive, hepatic, nec...","[<start>, 2, pieces, ,, subtotal, massive, hep...",9
3,GTEX-1117F-1226,Spleen,"2 pieces, marked congestion/autolysis",/project/GutIntelligenceLab/ss4yd/gtex_data/ac...,/project/GutIntelligenceLab/ss4yd/gtex_data/pa...,/project/GutIntelligenceLab/ss4yd/gtex_data/hi...,"[2, pieces, ,, marked, congestion/autolysis]","[<start>, 2, pieces, ,, marked, congestion/aut...",7
4,GTEX-1117F-1326,Adipose - Visceral (Omentum),"2 pieces, diffuse mesothelial hyperplasia; ~10...",/project/GutIntelligenceLab/ss4yd/gtex_data/ac...,/project/GutIntelligenceLab/ss4yd/gtex_data/pa...,/project/GutIntelligenceLab/ss4yd/gtex_data/hi...,"[2, pieces, ,, diffuse, mesothelial, hyperplas...","[<start>, 2, pieces, ,, diffuse, mesothelial, ...",16
...,...,...,...,...,...,...,...,...,...
1402,GTEX-13SLW-0726,Thyroid,"2 pieces, colloid cysts up to ~1.5mm, rep deli...",/project/GutIntelligenceLab/ss4yd/gtex_data/ac...,/project/GutIntelligenceLab/ss4yd/gtex_data/pa...,/project/GutIntelligenceLab/ss4yd/gtex_data/hi...,"[2, pieces, ,, colloid, cysts, up, to, ~1.5mm,...","[<start>, 2, pieces, ,, colloid, cysts, up, to...",13
1403,GTEX-13SLW-0826,Heart - Atrial Appendage,"2 pieces, no abnormalities",/project/GutIntelligenceLab/ss4yd/gtex_data/ac...,/project/GutIntelligenceLab/ss4yd/gtex_data/pa...,/project/GutIntelligenceLab/ss4yd/gtex_data/hi...,"[2, pieces, ,, no, abnormalities]","[<start>, 2, pieces, ,, no, abnormalities, <en...",7
1404,GTEX-13SLW-0926,Artery - Aorta,"6 pieces, atherosis up to ~0.5mm",/project/GutIntelligenceLab/ss4yd/gtex_data/ac...,/project/GutIntelligenceLab/ss4yd/gtex_data/pa...,/project/GutIntelligenceLab/ss4yd/gtex_data/hi...,"[6, pieces, ,, atherosis, up, to, ~0.5mm]","[<start>, 6, pieces, ,, atherosis, up, to, ~0....",9
1405,GTEX-13SLW-1026,Heart - Left Ventricle,"2 pieces, moderate ischemic changes/fibrosis, ...",/project/GutIntelligenceLab/ss4yd/gtex_data/ac...,/project/GutIntelligenceLab/ss4yd/gtex_data/pa...,/project/GutIntelligenceLab/ss4yd/gtex_data/hi...,"[2, pieces, ,, moderate, ischemic, changes/fib...","[<start>, 2, pieces, ,, moderate, ischemic, ch...",14


In [13]:
msk = np.random.rand(len(df)) < 0.9
# ndf=f.reset_index()
train = df[msk].copy()
temptest = df[~msk].copy()

tst_mask=np.random.rand(len(temptest)) < 0.5
test = temptest[tst_mask].copy()
val = temptest[~tst_mask].copy()

train['dtype']='train'
test['dtype']='test'
val['dtype']='val'

newndf=pd.concat([train,test,val])
newndf.dtype.value_counts()

train    1264
test       78
val        65
Name: dtype, dtype: int64

In [14]:
newndf['idx_tokens']=newndf.p_tokens.apply(lambda x: [word_map[i] if i in word_map.keys() else word_map['<unk>'] for i in x])
newndf

,pid,tissue_type,notes,svs_path,patch_path,reps_path,tokens,p_tokens,caplens,dtype,idx_tokens
1,GTEX-1117F-1026,Lung,"2 pieces, moderate congestion/moderate to mark...",/project/GutIntelligenceLab/ss4yd/gtex_data/ac...,/project/GutIntelligenceLab/ss4yd/gtex_data/pa...,/project/GutIntelligenceLab/ss4yd/gtex_data/hi...,"[2, pieces, ,, moderate, congestion/moderate, ...","[<start>, 2, pieces, ,, moderate, congestion/m...",10,train,"[649, 1, 2, 3, 6, 648, 7, 8, 9, 650, 0, 0, 0, ..."
2,GTEX-1117F-1126,Liver,"2 pieces, subtotal massive hepatic necrosis",/project/GutIntelligenceLab/ss4yd/gtex_data/ac...,/project/GutIntelligenceLab/ss4yd/gtex_data/pa...,/project/GutIntelligenceLab/ss4yd/gtex_data/hi...,"[2, pieces, ,, subtotal, massive, hepatic, nec...","[<start>, 2, pieces, ,, subtotal, massive, hep...",9,train,"[649, 1, 2, 3, 648, 648, 10, 11, 650, 0, 0, 0,..."
3,GTEX-1117F-1226,Spleen,"2 pieces, marked congestion/autolysis",/project/GutIntelligenceLab/ss4yd/gtex_data/ac...,/project/GutIntelligenceLab/ss4yd/gtex_data/pa...,/project/GutIntelligenceLab/ss4yd/gtex_data/hi...,"[2, pieces, ,, marked, congestion/autolysis]","[<start>, 2, pieces, ,, marked, congestion/aut...",7,train,"[649, 1, 2, 3, 8, 12, 650, 0, 0, 0, 0, 0, 0, 0..."
4,GTEX-1117F-1326,Adipose - Visceral (Omentum),"2 pieces, diffuse mesothelial hyperplasia; ~10...",/project/GutIntelligenceLab/ss4yd/gtex_data/ac...,/project/GutIntelligenceLab/ss4yd/gtex_data/pa...,/project/GutIntelligenceLab/ss4yd/gtex_data/hi...,"[2, pieces, ,, diffuse, mesothelial, hyperplas...","[<start>, 2, pieces, ,, diffuse, mesothelial, ...",16,train,"[649, 1, 2, 3, 13, 14, 15, 16, 17, 18, 648, 19..."
5,GTEX-1117F-1726,Pancreas,"2 pieces, marked saponification/autolysis, ~30...",/project/GutIntelligenceLab/ss4yd/gtex_data/ac...,/project/GutIntelligenceLab/ss4yd/gtex_data/pa...,/project/GutIntelligenceLab/ss4yd/gtex_data/hi...,"[2, pieces, ,, marked, saponification/autolysi...","[<start>, 2, pieces, ,, marked, saponification...",14,train,"[649, 1, 2, 3, 8, 23, 3, 648, 18, 24, 25, 3, 2..."
...,...,...,...,...,...,...,...,...,...,...,...
1322,GTEX-13PVR-1526,Small Intestine - Terminal Ileum,"6 pieces, well-preserved, lymphoid aggregares ...",/project/GutIntelligenceLab/ss4yd/gtex_data/ac...,/project/GutIntelligenceLab/ss4yd/gtex_data/pa...,/project/GutIntelligenceLab/ss4yd/gtex_data/hi...,"[6, pieces, ,, well-preserved, ,, lymphoid, ag...","[<start>, 6, pieces, ,, well-preserved, ,, lym...",16,val,"[649, 47, 2, 3, 367, 3, 147, 648, 481, 354, 18..."
1358,GTEX-13QJ3-1826,Esophagus - Muscularis,"6 pieces, all muscularis; good specimens",/project/GutIntelligenceLab/ss4yd/gtex_data/ac...,/project/GutIntelligenceLab/ss4yd/gtex_data/pa...,/project/GutIntelligenceLab/ss4yd/gtex_data/hi...,"[6, pieces, ,, all, muscularis, ;, good, speci...","[<start>, 6, pieces, ,, all, muscularis, ;, go...",10,val,"[649, 47, 2, 3, 230, 55, 16, 142, 164, 650, 0,..."
1363,GTEX-13QJC-0226,Heart - Left Ventricle,2 pieces; mild patchy scarring,/project/GutIntelligenceLab/ss4yd/gtex_data/ac...,/project/GutIntelligenceLab/ss4yd/gtex_data/pa...,/project/GutIntelligenceLab/ss4yd/gtex_data/hi...,"[2, pieces, ;, mild, patchy, scarring]","[<start>, 2, pieces, ;, mild, patchy, scarring...",8,val,"[649, 1, 2, 16, 101, 43, 582, 650, 0, 0, 0, 0,..."
1378,GTEX-13RTJ-2726,Muscle - Skeletal,"2 pieces, ~50% interstitial fat/edema, evidenc...",/project/GutIntelligenceLab/ss4yd/gtex_data/ac...,/project/GutIntelligenceLab/ss4yd/gtex_data/pa...,/project/GutIntelligenceLab/ss4yd/gtex_data/hi...,"[2, pieces, ,, ~50, %, interstitial, fat/edema...","[<start>, 2, pieces, ,, ~50, %, interstitial, ...",13,val,"[649, 1, 2, 3, 240, 18, 24, 648, 3, 200, 38, 2..."


In [15]:
newndf.to_pickle('prepared_prelim_data_tokenized.pickle')

In [16]:
!pip install "git+https://github.com/salaniz/pycocoevalcap.git"

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/salaniz/pycocoevalcap.git to /tmp/pip-req-build-x5i5239g
  ERROR: Error [Errno 2] No such file or directory: 'git' while executing command git version
ERROR: Cannot find command 'git' - do you have 'git' installed and in your PATH?
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [24]:
!pip install pycocoevalcap --user

  Using cached pycocoevalcap-1.2-py3-none-any.whl (104.3 MB)
  Using cached pycocotools-2.0.6.tar.gz (24 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  ERROR: Command errored out with exit status 1:
   command: /usr/local/bin/python /usr/local/lib/python3.8/site-packages/pip/_vendor/pep517/in_process/_in_process.py build_wheel /tmp/tmptxp4tb3y
       cwd: /tmp/pip-install-ercuzpbt/pycocotools_275e24637cfc4118ae5f0caec13088e1
  Complete output (20 lines):
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build/lib.linux-x86_64-cpython-38
  creating build/lib.linux-x86_64-cpython-38/pycocotools
  copying pycocotools/mask.py -> build/lib.linux-x86_64-cpython-38/pycocotools
  copying pycocotools/cocoeval.py -> build/lib.linux-x86_64-cpython-38/pycocotools
  copying pycocotools/coco.py -> build/lib.linux-x86_64-cpython-38/pycocotools
  copying pycocotools/__init__